In [1]:
import nltk
from nltk import word_tokenize
nltk.download('brown')
nltk.download('averaged_perceptron_tagger')

import matplotlib
%matplotlib inline


[nltk_data] Downloading package brown to
[nltk_data]     /usr4/cs591/shawnlin/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr4/cs591/shawnlin/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [4]:
import sys
!{sys.executable} -m pip install --user stanfordnlp

     |################################| 163kB 4.8MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import warnings

warnings.filterwarnings(action='once')

In [3]:
import spacy
import spacy_wordnet
from tqdm import tqdm
from collections import Counter
nlp = spacy.load("en_core_web_sm")
# nlp1 = spacy.load('en')
# nlp1.add_pipe(WordnetAnnotator(nlp1.lang), after='tagger')

/share/pkg.7/python3/3.6.9/install/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/usr4/cs591/shawnlin/.local/lib/python3.6/site-packages/thinc/neural/_custom_kernels.py:36: ResourceWarning: unclosed file <_io.TextIOWrapper name='/usr4/cs591/shawnlin/.local/lib/python3.6/site-packages/thinc/neural/_custom_kernels.cu' mode='r' encoding='utf8'>
  SRC = (PWD / "_custom_kernels.cu").open("r", encoding="utf8").read()
/usr4/cs591/shawnlin/.local/lib/python3.6/site-packages/thinc/neural/_custom_kernels.py:39: ResourceWarning: unclosed file <_io.TextIOWrapper name='/usr4/cs591/shawnlin/.local/lib/python3.6/site-packages/thinc/neural/_murmur3.cu' mode='r' encoding='utf8'>
  MMH_SRC = (PWD / "_murmur3.cu").open("r", encoding="utf8").read()


In [5]:
import stanfordnlp
# stanfordnlp.download('en')
stanford_nlp = stanfordnlp.Pipeline(use_gpu=True)
# doc = nlp("The elephant to the left.")
# doc.sentences[0].print_dependencies()

Use device: gpu
---
Loading: tokenize
With settings: 
{'model_path': '/usr4/cs591/shawnlin/stanfordnlp_resources/en_ewt_models/en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/usr4/cs591/shawnlin/stanfordnlp_resources/en_ewt_models/en_ewt_tagger.pt', 'pretrain_path': '/usr4/cs591/shawnlin/stanfordnlp_resources/en_ewt_models/en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/usr4/cs591/shawnlin/stanfordnlp_resources/en_ewt_models/en_ewt_lemmatizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings: 
{'model_path': '/usr4/cs591/shawnlin/stanfordnlp_resources/en_ewt_models/en_ewt_parser.pt', 'pretrain_path': '/usr4/cs591/

In [19]:
%%time
# doc = stanford_nlp("red duck in the front.")
doc = stanford_nlp("third cop car")
doc.sentences[0].print_dependencies()

('third', '3', 'amod')
('cop', '3', 'compound')
('car', '0', 'root')
CPU times: user 38.2 ms, sys: 731 µs, total: 38.9 ms
Wall time: 37.9 ms


In [26]:
import re
dependency = doc.sentences[0].dependencies_string().split("\n")

# def track_dependency()

def get_modifiers(sentence):
    
    modifier_chunks = {"nmod": None, "amod": None, "acl": None, "compound": None, "advmod": None}
    
    term_2_idx = {t._text: int(t._index) for t in sentence.tokens} #{: for t in sentence.tokens}
    term_2_idx["ROOT"] = 0
    #print(term_2_idx)
    
    from_to_map = {i+1: int(d[0]._index) for i, d in enumerate(sentence.dependencies)}
    to_from_map = {}
    for k, v in from_to_map.items():
        if v not in to_from_map:
            to_from_map[v] = [k]
        else:
            to_from_map[v].append(k)
    
    def idx_to_term(idx):
        return sentence.tokens[idx-1]._text

    #print(from_to_map)
    #print(to_from_map)
    assert(0 in to_from_map and len(to_from_map[0]) == 1)
    root_term_idx = to_from_map[0][0]
    root_term = idx_to_term(root_term_idx)
    #print("Root", root_term_idx, root_term)
    #print(to_from_map[root_term_idx])
    

    
    # Determine which relation it is?
    if root_term_idx not in to_from_map:
        return root_term, modifier_chunks

    # TODO: Merge compound nouns            
    
    for from_node in to_from_map[root_term_idx]:
        rel = sentence.dependencies[from_node-1][1]
        from_term = idx_to_term(from_node)
        #print(from_term, from_node, rel)
        
        # amod case, TODO: apply bfs too
        #if rel == "amod":
        #    to_append = from_term
        # nmod/acl case
        if rel in ["nmod", "acl", "amod", "compound", "advmod"]:
            chunk_span_ids = []
            
            # Apply BFS
            queue = [from_node]
            while len(queue) != 0:
                node = queue.pop(0)
                chunk_span_ids.append(node)
                for children in to_from_map.get(node, []):
                    queue.append(children)

            chunk_span_ids = sorted(chunk_span_ids)
            to_append = " ".join([idx_to_term(t_id) for t_id in chunk_span_ids])
        # other cases
        else:
            continue

        if modifier_chunks[rel] is not None:
            modifier_chunks[rel].append(to_append)
        else:
            modifier_chunks[rel] = [to_append]

    #print("Modifier chunks:", modifier_chunks)
    return root_term, modifier_chunks

get_modifiers(doc.sentences[0])

('elephant',
 {'nmod': None, 'amod': None, 'acl': None, 'compound': None, 'advmod': None})

In [45]:
import itertools
class_modifiers = {} # class: Counter()
modifiers = {} # rel: Counter()
cooccur = {}

for i, (sent, obj_type, img_url) in enumerate(zip(sents, obj_types, imgs)):
    doc = stanford_nlp(sent)
    
    # Lemming dict
    #lemma_dict = {token.text: token.lemma_.lower() for token in doc}
    if i == 200: break
    root_term, modifier_chunks = get_modifiers(doc.sentences[0])
    print("Sentence:", sent)
    doc.sentences[0].print_dependencies()
#     print("\tobj_type:", obj_type)
#     print("\troot_noun:", root_term)
    print("\tmodifiers:", modifier_chunks)
    
    # TODO: Check hyper/hyponym relation for obj_type and root_term
    if obj_type != root_term:
        continue
        
    for rel, chunk_list in modifier_chunks.items():
        if chunk_list is None:
            continue

        for chunk in chunk_list:
            if obj_type not in class_modifiers:
                class_modifiers[obj_type] = Counter([chunk])
            else:
                class_modifiers[obj_type].update([chunk])

            if rel not in modifiers:
                modifiers[rel] = Counter([chunk])
            else:
                modifiers[rel].update([chunk])

                    
    to_iterate = []
    for chunk_list in modifier_chunks.values():
        if chunk_list is not None:
            for chunk in chunk_list:
                to_iterate.append(chunk)

    for a, b in itertools.combinations(to_iterate, 2):
        
        a, b = sorted([a, b])
        if a not in cooccur:
            cooccur[a] = Counter([b])
        else:
            cooccur[a].update([b])
        if b not in cooccur:
            cooccur[b] = Counter([a])
        else:
            cooccur[b].update([a])


Sentence: center chair facing away
('center', '2', 'compound')
('chair', '0', 'root')
('facing', '2', 'acl')
('away', '3', 'advmod')
	modifiers: {'nmod': None, 'amod': None, 'acl': ['facing away'], 'compound': ['center'], 'advmod': None}
Sentence: middle chair
('middle', '2', 'amod')
('chair', '0', 'root')
	modifiers: {'nmod': None, 'amod': ['middle'], 'acl': None, 'compound': None, 'advmod': None}
Sentence: tan chair center
('tan', '2', 'compound')
('chair', '3', 'compound')
('center', '0', 'root')
	modifiers: {'nmod': None, 'amod': None, 'acl': None, 'compound': ['tan chair'], 'advmod': None}
Sentence: chair in fron right
('chair', '0', 'root')
('in', '4', 'case')
('fron', '4', 'case')
('right', '1', 'nmod')
	modifiers: {'nmod': ['in fron right'], 'amod': None, 'acl': None, 'compound': None, 'advmod': None}
Sentence: chair right in front
('chair', '0', 'root')
('right', '4', 'advmod')
('in', '4', 'case')
('front', '1', 'nmod')
	modifiers: {'nmod': ['right in front'], 'amod': None, 'a

Sentence: no problem top right corner person
('no', '2', 'det')
('problem', '0', 'root')
('top', '6', 'amod')
('right', '6', 'amod')
('corner', '6', 'compound')
('person', '2', 'appos')
	modifiers: {'nmod': None, 'amod': None, 'acl': None, 'compound': None, 'advmod': None}
Sentence: giraffe legs at bottom
('giraffe', '2', 'compound')
('legs', '0', 'root')
('at', '4', 'case')
('bottom', '2', 'nmod')
	modifiers: {'nmod': ['at bottom'], 'amod': None, 'acl': None, 'compound': ['giraffe'], 'advmod': None}
Sentence: giraffe legs
('giraffe', '2', 'compound')
('legs', '0', 'root')
	modifiers: {'nmod': None, 'amod': None, 'acl': None, 'compound': ['giraffe'], 'advmod': None}
Sentence: giraffe head
('giraffe', '2', 'compound')
('head', '0', 'root')
	modifiers: {'nmod': None, 'amod': None, 'acl': None, 'compound': ['giraffe'], 'advmod': None}
Sentence: gifafe
('gifafe', '0', 'root')
	modifiers: {'nmod': None, 'amod': None, 'acl': None, 'compound': None, 'advmod': None}
Sentence: giraffe looking a

Sentence: man on right
('man', '0', 'root')
('on', '3', 'case')
('right', '1', 'nmod')
	modifiers: {'nmod': ['on right'], 'amod': None, 'acl': None, 'compound': None, 'advmod': None}
Sentence: red guy
('red', '2', 'amod')
('guy', '0', 'root')
	modifiers: {'nmod': None, 'amod': ['red'], 'acl': None, 'compound': None, 'advmod': None}
Sentence: donut on the right
('do', '2', 'aux')
('nut', '0', 'root')
('on', '5', 'case')
('the', '5', 'det')
('right', '2', 'nmod')
	modifiers: {'nmod': ['on the right'], 'amod': None, 'acl': None, 'compound': None, 'advmod': None}
Sentence: donut bottom right
('do', '0', 'root')
('nut', '3', 'compound')
('bottom', '1', 'obj')
('right', '1', 'advmod')
	modifiers: {'nmod': None, 'amod': None, 'acl': None, 'compound': None, 'advmod': ['right']}
Sentence: right donut
('right', '2', 'amod')
('donut', '0', 'root')
	modifiers: {'nmod': None, 'amod': ['right'], 'acl': None, 'compound': None, 'advmod': None}
Sentence: top left donut
('top', '2', 'obl:npmod')
('left'

Sentence: man on left on phone
('man', '0', 'root')
('on', '3', 'mark')
('left', '1', 'acl')
('on', '5', 'case')
('phone', '3', 'obl')
	modifiers: {'nmod': None, 'amod': None, 'acl': ['on left on phone'], 'compound': None, 'advmod': None}
Sentence: white and blue person
('white', '4', 'amod')
('and', '3', 'cc')
('blue', '1', 'conj')
('person', '0', 'root')
	modifiers: {'nmod': None, 'amod': ['white and blue'], 'acl': None, 'compound': None, 'advmod': None}
Sentence: guy on bike farthest right
('guy', '0', 'root')
('on', '4', 'case')
('bike', '4', 'compound')
('farthest', '1', 'nmod')
('right', '1', 'advmod')
	modifiers: {'nmod': ['on bike farthest'], 'amod': None, 'acl': None, 'compound': None, 'advmod': ['right']}
Sentence: white shirt
('white', '2', 'amod')
('shirt', '0', 'root')
	modifiers: {'nmod': None, 'amod': ['white'], 'acl': None, 'compound': None, 'advmod': None}
Sentence: blue left
('blue', '2', 'amod')
('left', '0', 'root')
	modifiers: {'nmod': None, 'amod': ['blue'], 'acl'

Sentence: cow on right
('cow', '0', 'root')
('on', '3', 'case')
('right', '1', 'nmod')
	modifiers: {'nmod': ['on right'], 'amod': None, 'acl': None, 'compound': None, 'advmod': None}
Sentence: sleeping animal on left
('sleeping', '0', 'root')
('animal', '1', 'obj')
('on', '4', 'mark')
('left', '1', 'advcl')
	modifiers: {'nmod': None, 'amod': None, 'acl': None, 'compound': None, 'advmod': None}
Sentence: cow sleeping on left
('cow', '2', 'compound')
('sleeping', '0', 'root')
('on', '4', 'mark')
('left', '2', 'acl')
	modifiers: {'nmod': None, 'amod': None, 'acl': ['on left'], 'compound': ['cow'], 'advmod': None}
Sentence: left side
('left', '2', 'amod')
('side', '0', 'root')
	modifiers: {'nmod': None, 'amod': ['left'], 'acl': None, 'compound': None, 'advmod': None}
Sentence: cow looking at camera
('cow', '0', 'root')
('looking', '1', 'acl')
('at', '4', 'case')
('camera', '2', 'obl')
	modifiers: {'nmod': None, 'amod': None, 'acl': ['looking at camera'], 'compound': None, 'advmod': None}
S

In [43]:
merge_cntr = Counter()
for k, v in modifiers.items():
    merge_cntr = merge_cntr + v

with open("./modifers.txt", "w+") as f:
    for tup in merge_cntr.most_common(500):
        if tup[0] in cooccur:
            co_occur = ["(%s, %i)" % (t[0], t[1]) for t in cooccur[tup[0]].most_common(5)]
        else:
            co_occur = []
        f.write("%s (freq:%i) co_occur:[%s]\n" % (tup[0], tup[1], ", ".join(co_occur)))


In [39]:
with open("class_modifiers.txt", "w+") as f:
    for k, v in {k: v for k, v in class_modifiers.items()}.items():
        f.write("\nObject Type: %s\n" % k)
        relations = Counter(el for el in v.elements() if v[el] > 0).most_common(200)
        for tup in relations:
            f.write("\t%s (freq:%i)\n" % (tup[0], tup[1]))

In [22]:
sents = []
obj_types = []
imgs = []
for s in open("/projectnb/llamagrp/shawnlin/ref-exp-gen/ref-analysis/test_expr.txt", "r").readlines():
    toks = s.strip().split("\t")
    sents.append(toks[0])
    obj_types.append(toks[1])    
    imgs.append(toks[2])

print(sents[:5])
print(obj_types[:5])
print(imgs[:5])

['center chair facing away', 'middle chair', 'tan chair center', 'chair in fron right', 'chair right in front']
['chair', 'chair', 'chair', 'chair', 'chair']
['https://cs.stanford.edu/people/rak248/VG_100K_2/2405502.jpg', 'https://cs.stanford.edu/people/rak248/VG_100K/2328907.jpg', 'https://cs.stanford.edu/people/rak248/VG_100K_2/2405502.jpg', 'https://cs.stanford.edu/people/rak248/VG_100K_2/2405502.jpg', 'https://cs.stanford.edu/people/rak248/VG_100K_2/2405502.jpg']


/share/pkg.7/python3/3.6.9/install/lib/python3.6/site-packages/ipykernel_launcher.py:4: ResourceWarning: unclosed file <_io.TextIOWrapper name='/projectnb/llamagrp/shawnlin/ref-exp-gen/ref-analysis/test_expr.txt' mode='r' encoding='ANSI_X3.4-1968'>
  after removing the cwd from sys.path.


In [17]:
def print_raw_sent(sent):
    return " ".join([t[0] for t in sent])

In [18]:
obj_2_modifier = {}
modifier = []

for i, (sent, obj_type, img_url) in enumerate(tqdm(zip(sents, obj_types, imgs))):
    doc = nlp(sent)
#     print("%s, obj_type=%s, \n(%s)" % (sent, obj_type, img_url))
    
    # Lemming dict
    lemma_dict = {token.text: token.lemma_.lower() for token in doc}

    for chunk in doc.noun_chunks:
        
        h_noun = chunk.root.text
        lem_h_noun = lemma_dict[h_noun]
        lem_obj_type = lemma_dict.get(obj_type, obj_type)
        if lem_h_noun == lem_obj_type:
#             print("\tMatches referrent type!")
#             print("\tck_text:[%s(%i, %i)], root_text:[%s], root_dep:[%s] root_head:[%s]" % (chunk.text, chunk.start, chunk.end, lem_h_noun, chunk.root.dep_, chunk.root.head.text))
            for i in range(chunk.start, chunk.end):
                token = doc[i]
                tag = token.tag_
                if tag not in {"NN", "NNP"}:
#                     print("\t\t%s:%s" % (token, token.tag_))
                    if lem_h_noun not in obj_2_modifier:
                        obj_2_modifier[lem_h_noun] = {}
                    obj_2_modifier[lem_h_noun][str(token)+"-"+str(tag)]=True
#                     obj_2_modifier[lem_h_noun] = obj_2_modifier.get(lem_h_noun, set()).add()
                    modifier.append(token)
#     print("")
#     if i >= 10:
#         break
print(len(obj_2_modifier))
modifier_cntr = Counter(modifier)
print(len(modifier_cntr))

33348it [03:09, 175.62it/s]

61
5247


In [24]:
len(obj_2_modifier["chair"])

72

In [23]:
modifier_cntr.most_common()[:100]

[(middle, 1),
 (front, 1),
 (black, 1),
 (left, 1),
 (shorter, 1),
 (far, 1),
 (right, 1),
 (right, 1),
 (big, 1),
 (left, 1),
 (no, 1),
 (top, 1),
 (right, 1),
 (black, 1),
 (bottom, 1),
 (right, 1),
 (second, 1),
 (second, 1),
 (third, 1),
 (third, 1),
 (white, 1),
 (and, 1),
 (blue, 1),
 (lowest, 1),
 (right, 1),
 (front, 1),
 (top, 1),
 (big, 1),
 (right, 1),
 (the, 1),
 (top, 1),
 (small, 1),
 (sheep, 1),
 (sheep, 1),
 (standing, 1),
 (sheep, 1),
 (taller, 1),
 (far, 1),
 (right, 1),
 (darker, 1),
 (back, 1),
 (middle, 1),
 (the, 1),
 (baggage, 1),
 (bottom, 1),
 (right, 1),
 (front, 1),
 (second, 1),
 (right, 1),
 (dark, 1),
 (first, 1),
 (front, 1),
 (back, 1),
 (front, 1),
 (big, 1),
 (big, 1),
 (sitting, 1),
 (blond, 1),
 (far, 1),
 (right, 1),
 (the, 1),
 (right, 1),
 (right, 1),
 (middle, 1),
 (androgenous, 1),
 (the, 1),
 (top, 1),
 (the, 1),
 (bottom, 1),
 (the, 1),
 (first, 1),
 (stripped, 1),
 (red, 1),
 (far, 1),
 (right, 1),
 (third, 1),
 (dark, 1),
 (black, 1),
 (the,

In [14]:
from spacy import displacy
doc = nlp("Rats are various medium-sized, long-tailed rodents.")
displacy.render(doc, style="dep")

In [24]:
# Try NLTK parsing

# grammar = 'CHUNK: {<V.*> <TO> <V.*>}'
grammar = "NP: {<DT>?[<JJ>|<JJS>]*<NN>}"
cp = nltk.RegexpParser(grammar)
# brown = nltk.corpus.brown
# for i, sent in enumerate(brown.tagged_sents()):

for sent in sents:
    print(sent)
    tagged_sent = nltk.pos_tag(word_tokenize(sent))

    print(tagged_sent)
    tree = cp.parse(tagged_sent)
    print(tree)
    #jupyter_draw_nltk_tree(tree)
    for subtree in tree.subtrees():
        if subtree.label() == 'NP':
#             pass
            print("\t", subtree)


black chair by table
[('black', 'JJ'), ('chair', 'NN'), ('by', 'IN'), ('table', 'NN')]


ValueError: Transformation generated invalid chunkstring:
  {<JJ><NN>}<IN{><NN>}